In [1]:
# Exercise 1: Importing Libraries and Loading Data
import pandas as pd
import os

# Set the path to the dataset
path = r'C:\Users\luis\Desktop\Carrer Foundry boot camp\Python Fundamentals for Data Analysts\Data\Prepared Data'
file_name = 'ords_prods_merge_updated_4.8.pkl'
file_path = os.path.join(path, file_name)

# Load the dataframe
df = pd.read_pickle(file_path)

# Display the first few rows of the dataframe
print(df.head())


   order_id  user_id  order_number  order_dow  order_hour_of_day  \
0   2539329        1             1          2                  8   
1   2398795        1             2          3                  7   
2    473747        1             3          3                 12   
3   2254736        1             4          4                  7   
4    431534        1             5          4                 15   

   days_since_prior_order  product_id  add_to_cart_order  reordered eval_set  \
0                     NaN         196                  1          0    prior   
1                    15.0         196                  1          1    prior   
2                    21.0         196                  1          1    prior   
3                    29.0         196                  1          1    prior   
4                    28.0         196                  1          1    prior   

  product_name  aisle_id  department_id  prices  
0         Soda        77              7     9.0  
1         

In [2]:
# Number of rows in the ords_prods_merge_updated_4.8
ords_prods_merge_rows = df.shape[0]
print(f"Number of rows of the merged data is: {ords_prods_merge_rows}")

Number of rows of the merged data is: 32433030


In [2]:
# Exercise 2: Finding Aggregated Mean of 'order_number' Grouped by 'department_id'
# Group by 'department_id' and calculate the mean of 'order_number'
department_order_mean = df.groupby('department_id')['order_number'].mean()

# Display the results
print(department_order_mean)

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.215751
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170371
12    15.887622
13    16.583304
14    16.759763
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


# Exercise 3: Analyzing the Results
# The results for the entire dataframe provide a complete picture of the mean 'order_number' for each 'department_id',
# as opposed to the subset which only gives a partial view. This allows for more accurate analysis and insights.

In [3]:
# Exercise 4: Creating a Loyalty Flag for Existing Customers
import numpy as np

# Calculate the maximum order number for each user
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

# Create loyalty flag based on the max order number
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Display the frequency of the new 'loyalty_flag' column
print(df['loyalty_flag'].value_counts())

C:\Users\luis\AppData\Local\Temp\ipykernel_6292\3888536576.py:5: FutureWarning: The provided callable <function max at 0x000002912C62ADE0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)


loyalty_flag
Regular customer    15890458
Loyal customer      10293434
New customer         6249138
Name: count, dtype: int64


In [4]:
# Exercise 5: Checking Basic Statistics of Product Prices for Each Loyalty Category
# Basic statistics of product prices for each loyalty category
loyalty_price_stats = df.groupby('loyalty_flag')['prices'].describe()

# Display the statistics
print(loyalty_price_stats)


                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal customer    10293434.0  10.388854  327.868933  1.0  4.2  7.4  11.2   
New customer       6249138.0  13.294634  597.314116  1.0  4.2  7.4  11.3   
Regular customer  15890458.0  12.496419  539.488515  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal customer    99999.0  
New customer      99999.0  
Regular customer  99999.0  


In [5]:
# Exercise 6: Creating Spending Flag Based on Average Price
# Calculate the mean price for each user
df['avg_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)

# Create spending flag based on the average price
df['spending_flag'] = df['avg_price'].apply(lambda x: 'Low spender' if x < 10 else 'High spender')

# Display the first few rows of the dataframe with the new spending flag
print(df[['user_id', 'avg_price', 'spending_flag']].head())


C:\Users\luis\AppData\Local\Temp\ipykernel_6292\33158198.py:3: FutureWarning: The provided callable <function mean at 0x000002912C62B7E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df['avg_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)


   user_id  avg_price spending_flag
0        1   6.367797   Low spender
1        1   6.367797   Low spender
2        1   6.367797   Low spender
3        1   6.367797   Low spender
4        1   6.367797   Low spender


In [8]:
# Exercise 7: Creating Order Frequency Flag
# Calculate the median days_since_prior_order for each user
user_order_median = df.groupby('user_id')['days_since_prior_order'].median().reset_index()

# Rename the column for clarity
user_order_median.rename(columns={'days_since_prior_order': 'median_days_since_prior_order'}, inplace=True)

# Merge the median days_since_prior_order back to the original dataframe
df = df.merge(user_order_median, on='user_id', how='left')

# Create order frequency flag based on the median days_since_prior_order
def order_frequency_flag(median_days):
    if median_days > 20:
        return 'Non-frequent customer'
    elif 10 < median_days <= 20:
        return 'Regular customer'
    else:
        return 'Frequent customer'

df['order_frequency_flag'] = df['median_days_since_prior_order'].apply(order_frequency_flag)

# Display the first few rows of the dataframe with the new order frequency flag
print(df[['user_id', 'median_days_since_prior_order', 'order_frequency_flag']].head())



   user_id  median_days_since_prior_order   order_frequency_flag
0        1                           20.5  Non-frequent customer
1        1                           20.5  Non-frequent customer
2        1                           20.5  Non-frequent customer
3        1                           20.5  Non-frequent customer
4        1                           20.5  Non-frequent customer


In [9]:
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,eval_set,product_name,aisle_id,department_id,prices,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,prior,Soda,77,7,9.0,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,prior,Soda,77,7,9.0,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,prior,Soda,77,7,9.0,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,prior,Soda,77,7,9.0,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,prior,Soda,77,7,9.0,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [10]:
# Export the updated dataframe
output_file_name = 'ords_prods_merge_with_flags_task_4.8.pkl'
output_file_path = os.path.join(path, output_file_name)
df.to_pickle(output_file_path)
